In [1]:
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
import ipywidgets as widgets

%matplotlib inline

In [2]:
def convert_to_seconds(t_str):
    if isinstance(t_str, str):
        t = (datetime.strptime(t_str, '%M:%S.%f')).time()
        return ((t.hour * 60 + t.minute) * 60 + t.second + (t.microsecond / 1000000))
    return None

def convert_to_timestamp(secs):
    return str(timedelta(seconds=secs))

In [80]:
c2_log = pd.read_csv('data/LogCard.csv', header=3)

filter = (c2_log['Name'] == 'Bob') & (c2_log['Unnamed: 0'].isnull()) 
c2_log = c2_log[filter].copy()
c2_log = c2_log.dropna(axis=1, how='all').copy()
c2_log['Date'] = pd.to_datetime(c2_log['Date'])
c2_log['time_seconds'] = c2_log['Time'].apply(convert_to_seconds)
c2_log['time1_seconds'] = c2_log['Time.1'].apply(convert_to_seconds)

c2_log['workout_seq'] = c2_log.groupby(['Date', 'Time of Day', 'Workout Name']).cumcount() + 1

### Today's workout

In [81]:
today = datetime.now().date()
today

datetime.date(2018, 7, 15)

In [82]:
filter = (c2_log['Date'] == today) & (c2_log['Time.1'].isnull() == False)
c2_log[filter]['time1_seconds']

401    220.8
402    220.3
403    219.8
404    212.6
Name: time1_seconds, dtype: float64

In [83]:
filter = (c2_log['Date'] == today)
s = ' | ' + c2_log['Time.1']
c2_log[filter][['/500m']].join(s).to_clipboard(index=False)

In [84]:
# pyramid
filter = (c2_log['Date'] == today) & (c2_log['/500m'].isnull() == False)
#c2_log.loc[filter, ['Meters.1', '/500m', 'Time.1']].head(7).to_clipboard(index=False)
c2_log.loc[filter, ['Meters.1', '/500m']].to_clipboard(index=False)

In [85]:
# 3,000m, 2,500m, 2,000m
filter = (c2_log['/500m'].isnull() == False) | (c2_log['Time.1'].isnull() == False)
s = ' | ' + c2_log['Time.1']
c2_log[filter][['/500m']].join(s).head(4).to_clipboard(index=False)

### Past PP workouts

In [86]:
drp = widgets.Dropdown(options=['4x2000m/5:00r', '4x1000m/5:00r', '5x1500m/5:00r', '8x500m/3:30r', 'v250m/1:30r...7', 'v3000m/5:00r...3'])
drp

In [88]:
# '4x2000m/5:00r', '4x1000m/5:00r', '5x1500m/5:00r', '8x500m/3:30r', 'v250m/1:30r...7', 'v3000m/5:00r...3'
workout_name = drp.value 

filter = (c2_log['Workout Name'] == workout_name) & (c2_log['/500m'].isnull() == False) #& (c2_log['Meters.1'].isnull() == False)
pv = c2_log[filter][['Date', '/500m', 'Time.1', 'workout_seq']].pivot(columns='Date', index='workout_seq', values='/500m')
pv.columns = [d.date() for d in pv.columns]
pv

,2018-06-24,2018-07-15
workout_seq,,
1,01:51.1,01:49.1
2,01:53.6,01:50.4
3,01:53.7,01:50.1
4,01:52.8,01:49.9
5,01:44.4,01:46.3


In [89]:
pv.to_clipboard(index=False)

### PRs

In [9]:
# 06:45.8 2000 Bob Haffner 1:41.4
filter = c2_log['Workout Name'].isin(['100m', '500m', '1000m', '2000m', '5000m'])
cols = ['Date', 'Time', 'time_seconds']
c2_log[filter].groupby('Workout Name')[cols].apply(lambda x : x.nsmallest(n=1, columns='time_seconds')).sort_values('time_seconds')

,,Date,Time,time_seconds
Workout Name,,,,
100m,835,2015-09-06,00:16.8,16.8
500m,813,2015-09-13,01:24.8,84.8
1000m,800,2015-09-20,03:07.0,187.0
2000m,663,2017-02-12,06:50.4,410.4
5000m,1023,2013-06-03,18:50.4,1130.4
